<a href="https://colab.research.google.com/github/TurkuNLP/intro-to-nlp/blob/master/course_project_2023_template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to HLT Project (Template)

- Student(s) Name(s):
- Date:
- Chosen Corpus:
- Contributions (if group project):

### Corpus information

- Description of the chosen corpus:
- Paper(s) and other published materials related to the corpus:
- State-of-the-art performance (best published results) on this corpus:

---

## 1. Setup

In [22]:
# Your code to install and import libraries etc. here
import datasets 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import torch 
import transformers
import evaluate

---

## 2. Data download and preprocessing

### 2.1. Download the corpus

In [11]:
# Your code to download the corpus here
dset = datasets.load_dataset('imdb')
display(dset)


DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

### 2.2. Preprocessing

In [12]:
# Your code for any necessary preprocessing here
#Shuffle the dataset
dset = dset.shuffle(seed=42)
#Remove the unsupervised data part of the dataset as we dont need it for this task
del dset['unsupervised']

In [13]:
vectorizer = CountVectorizer(binary=True, max_features=25000)
text_list = [i['text'] for i in dset['train']]
vectorizer.fit(text_list)

def vectorize_example(examples, vectorizer): 
    vectorized = vectorizer.transform([examples["text"]])
    non_zero = vectorized.nonzero()[1]
    non_zero += 1
    return {'input_ids': non_zero}

# Conversion vocabulary 
idx2word = {v: k for k, v in vectorizer.vocabulary_.items()}

tokenized_data = dset.map(vectorize_example, num_proc=4, fn_kwargs={'vectorizer': vectorizer})

Map (num_proc=4): 100%|██████████| 25000/25000 [00:06<00:00, 4093.63 examples/s]


In [18]:
test_row = tokenized_data['train'][0]['input_ids']
convered_text = [idx2word[i] for i in test_row]
print(convered_text)

['above', 'action', 'actress', 'allah', 'americana', 'anders', 'area', 'argument', 'atari', 'beulah', 'bother', 'butch', 'bye', 'characterisation', 'clairvoyant', 'classical', 'compared', 'complicating', 'criminal', 'englishman', 'enjoyment', 'evaluated', 'factions', 'faraway', 'fur', 'goodbye', 'handbook', 'haven', 'howard', 'ifc', 'isaac', 'italian', 'judged', 'justice', 'languages', 'likeable', 'looming', 'maine', 'mayberry', 'moreau', 'noah', 'notable', 'onassis', 'oral', 'others', 'peoples', 'plotline', 'plotted', 'policeman', 'preferable', 'primed', 'quits', 'realm', 'relations', 'serio', 'similarity', 'simpler', 'spirited', 'spotlight', 'superficiality', 'suspected', 'thank', 'thatch', 'theater', 'thereafter', 'thick', 'things', 'thinker', 'tho', 'toad', 'took', 'violently', 'wayans', 'weak', 'weaken', 'weirdos', 'writings']


In [20]:
def collator(examples):
    batch = {"labels":torch.tensor(list(example["label"] for example in examples))}
    tensors = []
    max_len = max(len(example["input_ids"]) for example in examples)
    for example in examples:
        ids = torch.tensor(example["input_ids"])
        padded = torch.nn.functional.pad(ids, (0, max_len - ids.shape[0]))
        tensors.append(padded)
    batch["input_ids"] = torch.vstack(tensors)
    return batch

---

## 3. Machine learning model

### 3.1. Model training

In [23]:
# Your code to train the machine learning model on the training set and evaluate the performance on the validation set here

class MLPConfig(transformers.PretrainedConfig):
    pass
class MLP(transformers.PreTrainedModel):
    config_class=MLPConfig
    def __init__(self,config):
        super().__init__(config)
        self.vocab_size=config.vocab_size #embedding matrix row count
        self.embedding=torch.nn.Embedding(num_embeddings=self.vocab_size+1,embedding_dim=config.hidden_size,padding_idx=0)
        torch.nn.init.uniform_(self.embedding.weight.data,-0.001,0.001) 
        self.output=torch.nn.Linear(in_features=config.hidden_size,out_features=config.nlabels)

    def forward(self,input_ids,labels=None):
        embedded=self.embedding(input_ids)
        embedded_summed=torch.sum(embedded,dim=1)
        projected=torch.tanh(embedded_summed) 
        logits=self.output(projected)
        if labels is not None:
            loss=torch.nn.CrossEntropyLoss()
            return (loss(logits,labels),logits)
        else:
            return (logits,)

### 3.2 Hyperparameter optimization

In [24]:
# Your code for hyperparameter optimization here

mlp_config=MLPConfig(vocab_size=len(vectorizer.vocabulary_),hidden_size=20,nlabels=2)
mlp=MLP(mlp_config)

trainer_args = transformers.TrainingArguments(
    "mlp_checkpoints", #save checkpoints here
    evaluation_strategy="steps",
    logging_strategy="steps",
    eval_steps=500,
    logging_steps=500,
    learning_rate=1e-5, #learning rate of the gradient descent
    max_steps=20000,
    load_best_model_at_end=True,
    per_device_train_batch_size=128
)

accuracy = evaluate.load("accuracy")

def compute_accuracy(outputs_and_labels):
    outputs, labels = outputs_and_labels
    predictions = np.argmax(outputs, axis=-1) #pick the index of the "winning" label
    return accuracy.compute(predictions=predictions, references=labels)

mlp = MLP(mlp_config)


early_stopping = transformers.EarlyStoppingCallback(5)

trainer = transformers.Trainer(
    model=mlp,
    args=trainer_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"].select(range(5000)), #make a smaller subset to evaluate on
    compute_metrics=compute_accuracy,
    data_collator=collator,
    callbacks=[early_stopping]
)

trainer.train()

C:\Users\omarn\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\accelerate\accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
  2%|▎         | 500/20000 [00:08<05:28, 59.28it/s]

{'loss': 0.6663, 'grad_norm': 0.6451306343078613, 'learning_rate': 9.75e-06, 'epoch': 2.55}



  3%|▎         | 508/20000 [00:09<14:13, 22.83it/s]

{'eval_loss': 0.6431292295455933, 'eval_accuracy': 0.762, 'eval_runtime': 0.8556, 'eval_samples_per_second': 5843.516, 'eval_steps_per_second': 730.44, 'epoch': 2.55}


  5%|▌         | 1000/20000 [00:16<05:00, 63.21it/s]

...

100%|██████████| 20000/20000 [05:45<00:00, 65.01it/s]

{'loss': 0.2163, 'grad_norm': 0.36724454164505005, 'learning_rate': 0.0, 'epoch': 102.04}


                                                     
100%|██████████| 20000/20000 [05:46<00:00, 57.69it/s]

{'eval_loss': 0.3019106090068817, 'eval_accuracy': 0.8802, 'eval_runtime': 0.8354, 'eval_samples_per_second': 5984.855, 'eval_steps_per_second': 748.107, 'epoch': 102.04}
{'train_runtime': 346.6627, 'train_samples_per_second': 7384.701, 'train_steps_per_second': 57.693, 'train_loss': 0.31380066299438475, 'epoch': 102.04}


TrainOutput(global_step=20000, training_loss=0.31380066299438475, metrics={'train_runtime': 346.6627, 'train_samples_per_second': 7384.701, 'train_steps_per_second': 57.693, 'train_loss': 0.31380066299438475, 'epoch': 102.04})

In [25]:
#Save model 
#trainer.save_model("mlp_model")

### 3.3. Evaluation on test set

In [29]:
# Your code to evaluate the final model on the test set here
test_results = trainer.predict(tokenized_data["test"].select(range(5000)))
test_accuracy = compute_accuracy((test_results.predictions, test_results.label_ids))
print(f"Test accuracy: {test_accuracy}")

100%|██████████| 625/625 [00:00<00:00, 768.58it/s]

Test accuracy: {'accuracy': 0.8802}


In [42]:
#Convert the 10 first predictions to labels 
print("Predictions:", "\n", test_results.predictions[:10])
print("Binary predicted labels:", np.argmax(test_results.predictions[:10], axis=-1))


#Print first 10 true labels
true_labels = test_results.label_ids[:10]
print("True labels:", true_labels)


Predictions: 
 [[-1.4783843   1.2700074 ]
 [-0.16219756  0.06732692]
 [-0.01269698 -0.06483601]
 [-1.0602162   0.89951706]
 [ 0.37185746 -0.41186056]
 [-1.1537267   0.98373985]
 [ 0.09102534 -0.15580702]
 [ 1.1402507  -1.1289095 ]
 [ 0.29495418 -0.34836176]
 [-0.6786371   0.54820937]]
Binary predicted labels: [1 1 0 1 0 1 0 0 0 1]
True labels: [1 1 0 1 0 1 1 0 0 1]


---

## 4. Results and summary

### 4.1 Corpus insights

(Briefly discuss what you learned about the corpus and its annotation)

### 4.2 Results

(Briefly summarize your results)

### 4.3 Relation to state of the art

(Compare your results to the state-of-the-art performance)

---

## 5. Bonus Task (optional)

### 5.1. Annotating out-of-domain documents

(Briefly describe the chosen out-of-domain documents)

(Briefly describe the process of annotation)

### 5.2 Conversion into dataset

In [ ]:
# Your code to convert the annotations into a dataset here

### 5.3. Model evaluation on out-of-domain test set

In [ ]:
# Your code to evaluate the model on the out-of-domain test set here

### 5.4 Bonus task results

(Present the results of the evaluation on the out-of-domain test set)

### 5.5. Annotated data

In [ ]:
# Include your annotated out-of-domain data here